In [1]:
# Turn off unnecessary warnings
import warnings
warnings.filterwarnings("ignore")

# Import all the required packages
import json
import nltk
import urllib
import en_core_web_sm
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.wsd import lesk
from nltk.parse import CoreNLPParser
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import stopwords


# Performs Word tokenization on sentences
def Tokenization(sentence):
    tokens = nltk.word_tokenize(sentence)
    return tokens


# Performs Word Lemmatization
def Lemmatization(word_tokens):
    lemmas = []
    for token in word_tokens:
        lemmas.append(wordnet_lemmatizer.lemmatize(token))
    return lemmas


# Performs POS tagging
def POSTagging(word_tokens):
    POStags = nltk.pos_tag(word_tokens)
    return POStags   


# Obtains sentence heads
def getHeads(sentence, word_tokens):
    # Set up dependency parser
    dependencyParser = CoreNLPDependencyParser(url='http://localhost:9000')
    headList = []
    
    # Split the sentence
    stripedSen = sentence.strip(" '\"")
    if stripedSen != "":
        # Perform dependency parse
        depParse = dependencyParser.raw_parse(stripedSen)
        parseTree = list(depParse)[0]
        headWord = ""
        headWord = [k["word"] for k in parseTree.nodes.values() if k["head"] == 0][0]
        
        # Appends head if it's not empty
        if headWord != "":
            headList.append([headWord])
            
        # Obtain head word based on two cases
        else:
            for i, pp in enumerate(tagged):
                if pp.startswith("VB"):
                    headList.append([word_tokens[i]])
                    break
            if headWord == "":
                for i, pp in enumerate(tagged):
                    if pp.startswith("NN"):
                        headList.append([word_tokens[i]])
                        break
                        
    # For empty sentence, we just append "" as head
    else:
        headList.append([""])
 
    return headList


# Obtains WordNet Features
def WordNetFeatures(sentence, word_tokens):
    # Creates dictionaries for important word senses
    hypernyms_list = []
    hyponyms_list = []
    meronyms_list = []
    holonyms_list = []
    synonyms_list = []
    
    # Populates the above dictionaries according to the word senses associated with them
    for token in word_tokens:
        # Extracts best sense for each word using LESK
        best_sense = lesk(sentence, token)
        
        if best_sense is not None:
            # Obtains Synonyms
            synonym = token
            if best_sense. lemmas()[0].name() != token:
                synonym = best_sense.lemmas()[0].name()
            synonyms_list.append(synonym)
            
            # Obtains Hypernyms
            if best_sense.hypernyms() != []:
                hypernyms_list.append(best_sense.hypernyms()[0].lemmas()[0].name())
        
            # Obtains Hyponyms
            if best_sense.hyponyms() != []:
                hyponyms_list.append(best_sense.hyponyms()[0].lemmas()[0].name())
            
            # Obtains Meronyms
            if best_sense.part_meronyms() != []:
                meronyms_list.append(best_sense.part_meronyms()[0].lemmas()[0].name())
                
            # Obtains Holonyms
            if best_sense.part_holonyms() != []:
                holonyms_list.append(best_sense.part_holonyms()[0].lemmas()[0].name())
          
        # When there's no best sense, the token itself is the Synonym
        else:
            synonyms_list.append(token)
            
    return hypernyms_list, hyponyms_list, meronyms_list, holonyms_list, synonyms_list
   
    
# Performs Dependency Parsing
def DependencyParsing(sentence):
    dependencyParser = CoreNLPDependencyParser(url='http://localhost:9000')
    parse, = dependencyParser.raw_parse(sentence)
    
    # Dependency parsing to parse tree based patterns as features
    depParseResult = list(parse.triples())
    
    return depParseResult
    
    
# Main method
if __name__ == "__main__":
    # List of all article names in the repository
    articleNames = ["109.txt", "111.txt", "151.txt", "160.txt", "177.txt", 
                    "179.txt","181.txt", "196.txt", "199.txt", "220.txt", 
                    "222.txt", "226.txt", "288.txt", "297.txt", "304.txt", 
                    "342.txt", "347.txt", "360.txt", "390.txt", "400.txt", 
                    "56.txt", "58.txt", "6.txt"] 
    fileCount = len(articleNames)
    
    content = ""
    folderPath = "https://raw.githubusercontent.com/SaiManasaVedantam/NLP-QA-System-Datasets/main/Articles/"
    for i in range(fileCount):
        fileName = folderPath + articleNames[i]
        response = urllib.request.urlopen(fileName)
        webContents = response.read()
        stringTypeData = webContents.decode("utf-8")
        content = stringTypeData
        count = 0
        corpus_dict = {}

        # Obtain wordnet lemmatizer
        wordnet_lemmatizer = WordNetLemmatizer()

        # Get tokenized content
        sentences = []
        tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
        sentences.extend(tokenizer.tokenize(content))

        # Sentence count
        print("Total Sentences After splitting the document: ", len(sentences))
        print("Extracting features for each of the sentences and shown below:")
    
        # Extracting words
        for sen in sentences:
            print("\n------SENTENCE------")
            print(sen)

            print("\nWord Tokenization : Done")
            word_tokens = Tokenization(sen)
            #print(word_tokens)

            print("Word Lemmatization : Done")
            word_lemmas = Lemmatization(word_tokens)
            #print(word_lemmas)

            print("POS Tagging : Done")
            word_POStags = POSTagging(word_tokens)
            #print(word_POStags)

            print("WordNet Feature Extraction : Done")
            hypernyms, hyponyms, meronyms, holonyms, synonyms = WordNetFeatures(sen, word_tokens)
            #print(holonyms)
            
            print("Dependency Parsing : Done")
            depParse = DependencyParsing(sen)
            #print(depParse)

            print("Obtaining Heads : Done")
            headList = getHeads(sen, word_tokens)
            #print(headList)

            # Process data format to suit the Elastic Search requirements
            count = count + 1
            corpus_dict[count] = {}
            
            corpus_dict[count]["sentence"] = {}
            corpus_dict[count]["sentence"] = sen
            
            corpus_dict[count]["tokenized_text"] = {}
            corpus_dict[count]["tokenized_text"] = word_tokens
            
            corpus_dict[count]["lemma"] = {}
            corpus_dict[count]["lemma"] = word_lemmas
            
            corpus_dict[count]["tagged"] = {}
            corpus_dict[count]["tagged"] = word_POStags
            
            corpus_dict[count]["dependency_parse"] = {}
            corpus_dict[count]["dependency_parse"] = depParse
            
            corpus_dict[count]["synonyms"] = {}
            corpus_dict[count]["synonyms"] = synonyms
            
            corpus_dict[count]["hypernyms"] = {}
            corpus_dict[count]["hypernyms"] = hypernyms
            
            corpus_dict[count]["hyponyms"] = {}
            corpus_dict[count]["hyponyms"] = hyponyms
            
            corpus_dict[count]["meronyms"] = {}
            corpus_dict[count]["meronyms"] = meronyms
            
            corpus_dict[count]["holonyms"] = {}
            corpus_dict[count]["holonyms"] = holonyms
            
            corpus_dict[count]["head_word"] = {}
            corpus_dict[count]["head_word"] = headList[0]
            
            corpus_dict[count]["file_name"] = {}
            corpus_dict[count]["file_name"] = articleNames[i]

        output_name = '../Pipeline-Output/Parsed-' + articleNames[i]
        
        with open(output_name, 'w+', encoding='utf8') as output_file:
            json.dump(corpus_dict, output_file,  indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)

Total Sentences After splitting the document:  204
Extracting features for each of the sentences and shown below:

------SENTENCE------
Bird migration is the regular seasonal movement, often north and south along a flyway, between breeding and wintering grounds.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Many species of bird migrate.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Migration carries high costs in predation and mortality, including from hunting by humans, and is driven primarily by availability of food.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
It occurs mainly in the northern hemisp

Obtaining Heads : Done

------SENTENCE------
Non-migratory birds are said to be resident or sedentary.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Approximately 1800 of the world's 10,000 bird species are long-distance migrants.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Many bird populations migrate long distances along a flyway.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
The most common pattern involves flying north in the spring to breed in the temperate or Arctic summer and returning in the autumn to wintering grounds in warmer regions to the south.

Word Tokenization : Done
Word Lemmatizati

Obtaining Heads : Done

------SENTENCE------
Birds fly at varying altitudes during migration.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
An expedition to Mt.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Everest found skeletons of northern pintail Anas acuta and black-tailed godwit Limosa limosa at 5,000 m (16,000 ft) on the Khumbu Glacier.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Bar-headed geese Anser indicus have been recorded by GPS flying at up to 6,540 metres (21,460 ft) while crossing the Himalayas, at the same time engaging in the highest rates of climb to altitude for any bird.

Word To


------SENTENCE------
Prior to migration, 55 percent of their bodyweight is stored as fat to fuel this uninterrupted journey.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Seabird migration is similar in pattern to those of the waders and waterfowl.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Some, such as the black guillemot Cepphus grylle and some gulls, are quite sedentary; others, such as most terns and auks breeding in the temperate northern hemisphere, move varying distances south in the northern winter.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
The Arctic tern Sterna paradisaea has the long

Obtaining Heads : Done

------SENTENCE------
Many long-distance migrants appear to be genetically programmed to respond to changing day length.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Species that move short distances, however, may not need such a timing mechanism, instead moving in response to local weather conditions.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Thus mountain and moorland breeders, such as wallcreeper Tichodroma muraria and white-throated dipper Cinclus cinclus, may move only altitudinally to escape the cold higher ground.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Other spe

...


------SENTENCE------
Langlois notes that by October 1379, Mu Ying had allegedly captured 30,000 Tibetan prisoners and 200,000 domesticated animals.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Yet invasion went both ways; the Ming general Qu Neng, under the command of Lan Yu, was ordered to repel a Tibetan assault into Sichuan in 1390.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Discussions of strategy in the mid Ming dynasty focused primarily on recovery of the Ordos region, which the Mongols used as a rallying base to stage raids into Ming China.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Norbu


------SENTENCE------
The Karmapa launched a surprise ambush on Liu Yun's camp, seizing all the goods and valuables while killing or wounding half of Liu Yun's entire escort.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
After this fiasco, Liu fled for his life, but only returned to Chengdu several years later to find that the Zhengde Emperor had died.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Elliot Sperling, a specialist of Indian studies and the director of the Tibetan Studies program at Indiana University’s Department of Central Eurasia Studies, writes that "the idea that Tibet became part of China in the 13th century is a very recent construction."

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
Wor


------SENTENCE------
Klieger writes that Altan Khan's presence in the west effectively reduced Ming influence and contact with Tibet.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
After Altan Khan made peace with the Ming dynasty in 1571, he invited the third hierarch of the Gelug—Sönam Gyatso (1543–1588)—to meet him in Amdo (modern Qinghai) in 1578, where he accidentally bestowed him and his two predecessors with the title of Dalai Lama—"Ocean Teacher".

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
The full title was "Dalai Lama Vajradhara", "Vajradhara" meaning "Holder of the Thunderbolt" in Sanskrit.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Do

Obtaining Heads : Done

------SENTENCE------
Chen states that the fourth Dalai Lama Yonten Gyatso was granted the title "Master of Vajradhara" and an official seal by the Wanli Emperor in 1616.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
This was noted in the Biography of the Fourth Dalai Lama, which stated that one Soinam Lozui delivered the seal of the Emperor to the Dalai Lama.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
The Wanli Emperor had invited Yonten Gyatso to Beijing in 1616, but just like his predecessor he died before being able to make the journey.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTEN

Obtaining Heads : Done

------SENTENCE------
Although Güshi Khan had granted the Dalai Lama "supreme authority" as Goldstein writes, the title of 'King of Tibet' was conferred upon Güshi Khan, spending his summers in pastures north of Lhasa and occupying Lhasa each winter.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Van Praag writes that at this point Güshi Khan maintained control over the armed forces, but accepted his inferior status towards the Dalai Lama.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done
Dependency Parsing : Done
Obtaining Heads : Done

------SENTENCE------
Rawski writes that the Dalai Lama shared power with his regent and Güshi Khan during his early secular and religious reign.

Word Tokenization : Done
Word Lemmatization : Done
POS Tagging : Done
WordNet Feature Extraction : Done